# Accidentes de transito INE

## Objetivos

- Como mejorar el data set para prevenir accidentes de transito

### Para lograr el objetivo necesitamos

- Descubrir cuales son las causas de los accidentes de transito
- Identificar que informacion es necesaria para identificar las causas
- Descubrir como se previenen los accidentes de transito

# Causas de los accidentes de transito

Segun aseguradora mapfre las principales causas de accidentes de transito son:
 1. Fallas mecanicas
 2. Conductores ebrios
 3. Sueño
 4. Uso de telefono
 5. Exceso de velocidad
 6. Pasar el semáforo en rojo / Alto
 7. Distracciones
 
Ademas la organizacion mundial de la salud agrega como un factor para los accidentes de transito las infraestructura vial.


https://www.who.int/es/news-room/fact-sheets/detail/road-traffic-injuries 
https://www.mapfre.com.pe/viviendo-en-confianza/seguridad-vial/7-principales-causas-accidentes-auto/


# Datos de investigacion

Datos proveeidos por INE https://www.ine.gob.gt/estadisticasine/index.php/usuario/hechos_transito_menu del año 2009 a 2019

### Que datos se incluyen en los datos de la INE:

- Número de correlativo
- Año de ocurrencia
- Día de ocurrencia
- Hora de ocurrencia
- Grupo de hora
- Grupo de hora mañana, tarde y noche
- Mes de ocurrencia
- Día de la semana de ocurrencia
- Departamento de ocurrencia
- Municipio de ocurrencia
- Zona de ocurrencia
- Tipo de vehículo
- Marca del vehículo
- Color del vehículo
- Modelo del vehículo
- Grupo modelo del vehículo
- Tipo de evento

### Que datos son solicitados por las aseguradoras

- Datos del vehiculo asegurado:
    - Marca del vehiculo
    - Tipo del vehiculo
    - Modelo del vehiculo
    - Color del vehiculo
- Datos del accidente
    - Edad del conductor
    - Fecha
    - Hora
    - Lado de la calle (izquierdo/derecho)
    - Velocidad
    - Direccion en la que se traslada (vehiculos involucrados en el accidente)
    - Causa del accidente
    - Vehiculo que circulaba con preferencia de via

# Limpieza de datos

TODO:
- Juntar todos los años
- Mejorar limpieza

In [5]:
import krangl.*

In [6]:
val events = DataFrame.readCSV("hechos-transito-2019.csv")

In [9]:
events.head(10)

núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve
1,2019,1,5,1,1,1,2,1,101,3,4,64,5,2018,5,6
2,2019,1,21,4,3,1,2,1,101,1,2,90,14,9999,99,1
3,2019,1,7,2,1,1,2,1,101,18,1,41,8,2019,5,1
4,2019,1,11,2,1,1,2,1,102,99,1,69,5,2007,4,1
5,2019,1,8,2,1,1,2,1,101,7,4,27,1,9999,99,6
6,2019,1,0,1,1,1,2,1,101,7,1,999,99,9999,99,1
7,2019,1,4,1,1,1,2,1,115,99,4,999,1,9999,99,6
8,2019,1,4,1,1,1,2,1,114,99,1,999,99,9999,99,1
9,2019,1,11,2,1,1,2,1,108,99,1,69,99,9999,99,1
10,2019,1,12,3,2,1,2,1,110,99,1,56,99,9999,99,1


#### Verifiquemos que los datos tengan sentido

In [38]:
events["día_ocu"].values().distinct() // No hay fecha con mes con mas de 31 dias o con datos invalidos

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

In [39]:
events["mes_ocu"].values().distinct() // No hay mes mayor al 12 (enero 1 ... diciembre 12)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [44]:
events["hora_ocu"].values().distinct().sortedBy { it as? Int } // Hora entre 0 y 23 hrs

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [45]:
events["g_hora_5"].values().distinct() // Sola hay 3 grupos de hora

[1, 3, 2]

In [47]:
events["día_sem_ocu"].values().distinct().sortedBy { it as? Int } // Soloa hay 7 dias a la semana (1 ...7 )

[1, 2, 3, 4, 5, 6, 7]

In [50]:
events["depto_ocu"].values().distinct().sortedBy { it as? Int } // Solo hay 22 departamentos (1 ... 22)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [52]:
events["modelo_veh"].values().distinct().sortedBy { it as? Int } // Los modelos de vehiculos son validos

[1974, 1975, 1978, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 9999]

#### Busquemos datos vacios

In [96]:
// Columnas con valores NA
events.cols
    .filter { row -> row.isNA().any { it } }
    .map { it.name }

[]

## Vamos a convertir los codigos en los valores correspondientes

#### Primero definamos los valores por los que se reemplazaran los ids

In [157]:
val hourGroupNames = dataFrameOf("g_hora_5", "Grupo de hora (Nombre)")(
    1, "manana",
    2, "tarde",
    3, "noche"
)

In [158]:
val hourGroups = dataFrameOf("g_hora", "Grupo de hora (Rango)")(
    1, "00:00 a 05:59",
    2, "06:00 a 11:59",
    3, "12:00 a 17:59",
    4, "18:00 a 23:59"
)

In [159]:
val occurrenceMonth = dataFrameOf("mes_ocu", "Mes de ocurrencia")(
    1, "Enero",
    2, "Febrero",
    3, "Marzo",
    4, "Abril",
    5, "Mayo",
    6, "Junio",
    7, "Julio",
    8, "Agosto",
    9, "Septiembre",
    10, "Octubre",
    11, "Noviembre",
    12, "Diciembre"
)

In [160]:
val occurrenceDayOfTheWeek = dataFrameOf("día_sem_ocu", "Día de la semana de ocurrencia")(
    1, "Lunes",
    2, "Martes",
    3, "Miércoles",
    4, "Jueves",
    5, "Viernes",
    6, "Sábado",
    7, "Domingo"
)

In [155]:
val occurrenceDepartment = DataFrame.readCSV("depto_ocu.csv")

In [ ]:
val occurrenceDepartment = DataFrame.readCSV("depto_ocu.csv")

In [161]:
events
    .innerJoin(hourGroups, by = listOf("g_hora"))
    .innerJoin(hourGroupNames, by = listOf("g_hora_5"))
    .innerJoin(occurrenceMonth, by = listOf("mes_ocu"))
    .innerJoin(occurrenceDayOfTheWeek, by = listOf("día_sem_ocu"))
    .innerJoin(occurrenceDepartment, by = listOf("depto_ocu"))
    .remove("g_hora_5", "g_hora", "mes_ocu", "día_sem_ocu", "depto_ocu")
    .head(5)

núm_corre,año_ocu,día_ocu,hora_ocu,mupio_ocu,zona_ocu,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve,Grupo de hora (Rango),Grupo de hora (Nombre),Mes de ocurrencia,Día de la semana de ocurrencia,Occurrence department
199,2019,14,3,101,13,4,27,1,9999,99,6,00:00 a 05:59,manana,Enero,Lunes,Guatemala
531,2019,21,5,101,9,1,999,99,9999,99,5,00:00 a 05:59,manana,Enero,Lunes,Guatemala
198,2019,14,8,105,99,4,3,5,2018,5,1,06:00 a 11:59,manana,Enero,Lunes,Guatemala
528,2019,21,10,101,13,4,3,1,2017,5,5,06:00 a 11:59,manana,Enero,Lunes,Guatemala
529,2019,21,7,115,99,1,28,2,9999,99,5,06:00 a 11:59,manana,Enero,Lunes,Guatemala
